# Домашнее задание №2

В этой домашке вам будет нужно найти тексты на русском языке (размер корпуса не менее 200 слов), в которых будут какие-то трудные или неоднозначные для POS теггинга моменты и разметить их вручную – с помощью этих текстов мы будем оценивать качество работы наших теггеров. В текстах размечаем только части речи, ничего больше!

(1 балл) Создание, разметка корпуса и объяснение того, почему этот текст подходит для оценки (какие моменты вы тут считаете трудными для автоматического посттеггинга и почему, в этом вам может помочь второй ридинг). Не забывайте, что разные теггеры могут использовать разные тегсеты: напишите комментарий о том, какой тегсет вы берёте для разметки и почему.

In [134]:
#импортируем нужные библиотеки
import pandas as pd
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
from pymystem3 import Mystem
m = Mystem()
from pprint import pprint
import spacy
from nltk.tokenize import word_tokenize
nlp = spacy.load("ru_core_news_sm")

В корпус я включила слова с неизвестными словарю корнем, но образованные с помощью продуктивных аффиксов, авторские "придуманные слова", сложные слова, у которых вторая часть совпадает со словами в словаре Зализняка, слова с неизвестными корнями без продуктивных суффиксов, редкие и нестандартные формы, которые признаются ненормативными или окказиональными в языке и аббвериатуры. Для разметки я использовала граммемы, которые используются в pymorphy2 для русского языка, в pymorphy2 используются словари OpenCoprora и граммемы, принятые в OpenCorpora с небольшими изменениями. 

In [131]:
korpus = pd.read_csv('final_korpus.csv', sep = ';')

In [ ]:
pos_taggers = []
for pos_tager in korpus['POS']:
    pos_taggers.append(pos_tager)

pos_taggers

# Второе и Третье задание 
Потом вам будет нужно взять три POS теггера для русского языка (udpipe, stanza, natasha, pymorphy, mystem, spacy, deeppavlov) и «прогнать» текст через каждый из них.Затем оценим accuracy для каждого теггера. Заметьте, что в разных системах имена тегов и части речи могут отличаться, – вам надо будет свести это всё к единому стандарту с помощью какой-то функции-конвертера и сравнить с вашим размеченным руками эталоном - тоже с помощью какого-то кода или функции.

In [135]:
# spacy pos tagger
spacy_taggers = []
for line in korpus['Word']:
    spacy_taggers.append(nlp(line)[0].pos_)

In [136]:
for i, n in enumerate(spacy_taggers):
    if n == 'ADJ':
        spacy_taggers[i] = 'ADJF'
    if n == 'ADV':
        spacy_taggers[i] = 'ADVB'
    if n == 'PROPN':
        spacy_taggers[i] = 'NOUN'

In [137]:
import numpy
import sklearn.metrics

r = sklearn.metrics.confusion_matrix(pos_taggers, spacy_taggers)
r = numpy.flip(r)

acc = (r[0][0] + r[-1][-1]) / numpy.sum(r)
print(acc)

0.30288461538461536


In [138]:
# Mystem pos tagger 
mystem_korpus = []
for line in korpus['Word']:
    ana = m.analyze(line)
    mystem_korpus.append(ana)

mystem_korpus[:10]

[[{'analysis': [{'lex': 'ультраженственный',
     'wt': 0.4541687794,
     'qual': 'bastard',
     'gr': 'A=(пр,ед,полн,жен|дат,ед,полн,жен|род,ед,полн,жен|твор,ед,полн,жен)'}],
   'text': 'ультраженственной'},
  {'text': '\n'}],
 [{'analysis': [{'lex': 'миллионометр',
     'wt': 1,
     'qual': 'bastard',
     'gr': 'S,муж,неод=род,ед'}],
   'text': 'миллионометра'},
  {'text': '\n'}],
 [{'analysis': [{'lex': 'гильоше',
     'wt': 0.2741620489,
     'qual': 'bastard',
     'gr': 'S,сред,неод=(пр,мн|пр,ед|вин,мн|вин,ед|дат,мн|дат,ед|род,мн|род,ед|твор,мн|твор,ед|им,мн|им,ед)'}],
   'text': 'гильоше'},
  {'text': '\n'}],
 [{'analysis': [{'lex': 'полуколебание',
     'wt': 0.5196742212,
     'qual': 'bastard',
     'gr': 'S,сред,неод=род,мн'}],
   'text': 'полуколебаний'},
  {'text': '\n'}],
 [{'analysis': [{'lex': 'баухаус',
     'wt': 1,
     'qual': 'bastard',
     'gr': 'S,муж,неод=(вин,ед|им,ед)'}],
   'text': 'баухаус'},
  {'text': '\n'}],
 [{'analysis': [{'lex': 'скелетонный',
   

In [139]:
mystem_taggers = []
for word in mystem_korpus:
    if word[0]['analysis'] == []:
        mystem_taggers.append('None')
    else:
        gr = word[0]['analysis'][0]['gr']
        pos = gr.split('=')[0].split(',')[0]
        mystem_taggers.append(pos)

In [140]:
for i, n in enumerate(mystem_taggers):
    if n == 'A':
        mystem_taggers[i] = 'ADJF'
    if n == 'ADV':
        mystem_taggers[i] = 'ADVB'
    if n == 'S':
        mystem_taggers[i] = 'NOUN'
    if n == 'V':
        mystem_taggers[i] = 'VERB'
    if n == 'PART':
        mystem_taggers[i] = 'PRCL'

In [141]:
import numpy
import sklearn.metrics

r = sklearn.metrics.confusion_matrix(pos_taggers, mystem_taggers)
r = numpy.flip(r)

acc = (r[0][0] + r[-1][-1]) / numpy.sum(r)
print(acc)

0.33653846153846156


In [142]:
# pymorphy pos tagger 
pymorphy_taggers = []
for word in korpus['Word']:
    if morph.parse(word)[0].tag.POS == None:
        pymorphy_taggers.append('None')
    else:
        pymorphy_taggers.append(morph.parse(word)[0].tag.POS)

In [143]:
import numpy
import sklearn.metrics

r = sklearn.metrics.confusion_matrix(pos_taggers, pymorphy_taggers)
r = numpy.flip(r)

acc = (r[0][0] + r[-1][-1]) / numpy.sum(r)
print(acc)

0.1971153846153846


Мы использовали теггеры pymorphy, pymystem и spacy. Из всех трех теггеров accuracy больше всех набрал теггер pymystem. Достоинства Mystem'a: хорошее качество разбора, по умолчанию разрешается частеречная омонимия (внутри части речи остается), при разборе учитывается контекст
и совместим с разметкой НКРЯ

# Четвертое задание 

Дальше вам нужно взять лучший теггер для русского языка и с его помощью написать функцию (chunker), которая выделяет из размеченного текста 3 типа n-грамм, соответствующих какому-то шаблону (к примеру не + какая-то часть речи или NP или сущ.+ наречие и тд) В предыдущем дз многие из вас справедливо заметили, что если бы мы могли класть в словарь не только отдельные слова, но и словосочетания, то программа работала бы лучше. Предложите 3 шаблона (слово + POS-тег / POS-тег + POS-тег) запись которых в словарь, по вашему мнению, улучшила бы качество работы программы из предыдущей домашки. Балл за объяснение того, почему именно эти группы вы взяли, балл за создание такого рода чанкера, балл за за встраивание функции в программу из предыдущей домашки, балл за сравнение качества предсказания тональности с улучшением и без.

In [2]:
# возьмем датасет из прошлого домашнего задания, в той тетрадке я сохраняла этот файл и вызывается как selected, он уже есть на гитхабе
selected = pd.read_csv('comments.csv', sep=',')

In [3]:
#разделим на положительные и отрицательные
positive_df = selected[selected['label_binary'] == 1.0]
negative_df = selected[selected['label_binary'] == 0.0]

# Функция chunking
Напишем функцию, которая будет выделять три типа словосочетаний, мы возьмем три шаблона: прилагательные и существительные, глагол и наречие, существительное и существительное. Прилагательное описывает существительное, признак предмета, в нем может содержаться положительная или негативная оценка, наречие тоже описывает признак действия, существительное тоже может иметь негативную или положительную оценку. Так как среди остальных теггеров лучше себя показал теггер pymystem, то используем его в этой функции и вытащим с помощью него части речи. Обязательно будем проверять, есть ли "analysis" в разборе, потому что может и не быть. Проверяем у каждого слова в предложении, есть ли его часть речи в нашем списке ent. Если нету, то добавляем в chunked_sent, а если есть, то смотрим составляет это слова одну из наших шаблонов. После этого добавляем в chunked_sent данные лексемы. 

In [4]:
sent = 'Очень богатый мужчина не дал хорошо мало золотых монет туртсияхх ибиптахучить бедному мальчику'
ent = ("A", "V", "S")
def chunking(sent):
    tagged = []
    chunk = []
    chunked_sent = []
    ana = m.analyze(sent)
    for word in ana:
        if "analysis" in word:
            if word['analysis'] == []:
                chunked_sent.append(word['text'])
            else:
                tagged.append(word['analysis'][0])


    for i, n in enumerate(tagged):
        if i < (len(tagged) - 1):
            if  tagged[i]['gr'].split('=')[0].split(',')[0] not in ent:
                chunked_sent.append(tagged[i]['lex'])
            else:
                if tagged[i]['gr'].split('=')[0].split(',')[0] == "A":
                    if tagged[i+1]['gr'].split('=')[0].split(',')[0] == "S":
                        chunk.append(tagged[i]['lex'])
                        chunk.append(tagged[i+1]['lex'])
                        chunked_sent.append(chunk)
                        chunk = []
                elif tagged[i]['gr'].split('=')[0].split(',')[0] == "V":
                    if tagged[i+1]['gr'].split('=')[0].split(',')[0] == "ADV":
                        chunk.append(tagged[i]['lex'])
                        chunk.append(tagged[i+1]['lex'])
                        chunked_sent.append(chunk)
                        chunk = []
                elif tagged[i]['gr'].split('=')[0].split(',')[0] == "S":
                    if tagged[i+1]['gr'].split('=')[0].split(',')[0] == "S":
                        chunk.append(tagged[i]['lex'])
                        chunk.append(tagged[i+1]['lex'])
                        chunked_sent.append(chunk)
                        chunk = []
    return chunked_sent

In [5]:
#применим функцию chunking на нашем датасете
positive_tokens = []
for line in positive_df['lemm_clean_tex']:
    positive_tokens.append(chunking(line))

negative_tokens = []
for line in negative_df['lemm_clean_tex']:
    negative_tokens.append(chunking(line))

In [13]:
import operator
from functools import reduce
positive_tokens = reduce(operator.concat, positive_tokens)
negative_tokens = reduce(operator.concat, negative_tokens)

In [31]:
positive_tuple = []
for i in positive_tokens:
    if type(i) is list:
        positive_tuple.append(tuple(i))
    else:
        positive_tuple.append(i)

negative_tuple = []
for i in negative_tokens:
    if type(i) is list:
        negative_tuple.append(tuple(i))
    else:
        negative_tuple.append(i)

In [34]:
#Возвращает словарь {слово: абсолютная частота встречаемости} на основе списка слов positive_tokens
pos_freq = {}
for pos_word in positive_tuple:
    if pos_word in pos_freq:
        pos_freq[pos_word] += 1
    else:
        pos_freq[pos_word] = 1

neg_freq = {}
for neg_word in negative_tuple:
    if neg_word in neg_freq:
        neg_freq[neg_word] += 1
    else:
        neg_freq[neg_word] = 1

In [37]:
#выбросим слова, которые встречаются только 1-2 раза
pos_words = []
for pos_word, freq_word in pos_freq.items():
    if freq_word > 2:
        pos_words.append(pos_word)

neg_words = []
for neg_word, freq_word in neg_freq.items():
    if freq_word > 2:
        neg_words.append(neg_word)

In [38]:
pos_set = set(pos_words)
neg_set = set(neg_words)

In [41]:
diff_pos = pos_set.difference(neg_set)
diff_neg = neg_set.difference(pos_set)

In [ ]:
diff_pos = pos_set.difference(neg_set)
diff_neg = neg_set.difference(pos_set)
print(diff_pos)
print(diff_neg)

Возьмем функцию из прошлого домашнего задания и немного ее переделаем под наше задание, используем в ней нашу функцию chunking и списки заменим на кортежи. 

In [70]:
def sentiment_analysis(reply):
    clear_text = re.sub(r'[^А-яЁё]+', ' ', text).lower()
    lemmas = m.lemmatize(clear_text)
    lemm_text = ''.join(lemmas)
    tokens = word_tokenize(lemm_text)
    joined_tokens= ' '.join(tokens)
    chunked = chunking(joined_tokens)
    sent_tuple = []
    for i in chunked:
        if type(i) is list:
            sent_tuple.append(tuple(i))
        else:
            sent_tuple.append(i)

    sample_set = set(tokens)
    if sample_set.intersection(diff_pos) > sample_set.intersection(diff_neg):
        print('positive')
    else:
        print('negative')

In [72]:
#вытащим первые пять положительных и отрицательных комментариев, чтобы проверить функцию и рассчитать accuracy
scrapped = {}
for line in positive_df['text'].head():
    scrapped[line] = "positive"

for line in negative_df['text'].head():
    scrapped[line] = "negative"

In [73]:
y_true = list(scrapped.values())
scrapped_keys = list(scrapped.keys())

In [77]:
b = []
def sentiment_analysis(reply):
    clear_text = re.sub(r'[^А-яЁё]+', ' ', text).lower()
    lemmas = m.lemmatize(clear_text)
    lemm_text = ''.join(lemmas)
    tokens = word_tokenize(lemm_text)
    joined_tokens= ' '.join(tokens)
    chunked = chunking(joined_tokens)
    sent_tuple = []
    for i in chunked:
        if type(i) is list:
            sent_tuple.append(tuple(i))
        else:
            sent_tuple.append(i)

    sample_set = set(tokens)
    if sample_set.intersection(diff_pos) > sample_set.intersection(diff_neg):
        b.append('positive')
    else:
        b.append('negative')
    return b

In [78]:
for key in scrapped_keys:
    y_pred = sentiment_analysis(key)

In [79]:
import numpy
import sklearn.metrics


r = sklearn.metrics.confusion_matrix(y_true, y_pred)
r = numpy.flip(r)

acc = (r[0][0] + r[-1][-1]) / numpy.sum(r)
print(acc)

0.5


Получаем, что accuracy равно 0.5 как и было в прошлом задании. Также можем взять NLTK VADER и с помощью него, разделить слова и словосочетания в нашем словаре на положительные, нейтральные и отрицательные. 

In [ ]:
extraPhrase = positive_tokens + negative_tokens
extraPhrase

In [87]:
first_five = extraPhrase[:10]
first_five

['некоторые',
 ['список', 'любовь'],
 ['герой', 'р'],
 ['растрелять', 'правда'],
 'правда',
 ['воевать', 'недавно'],
 'недавно',
 ['неплохой', 'репортаж'],
 ['наградной', 'лист'],
 ['лист', 'бой']]

In [158]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer() 
def sentimentWordsFunct(x):
    senti_list_temp = []    
    for i in x:
        if type(i) is list:
            y = ' '.join(i)
        else:
            y = ''.join(i) 

        vs = analyzer.polarity_scores(y)
        senti_list_temp.append((y, vs))
        senti_list_temp = [w for w in senti_list_temp if w]    
        sentiment_list  = []
    for j in senti_list_temp:
        first = j[0]
        second = j[1]
    
        for (k,v) in second.items():
            if k == 'compound':
                if v < 0.0:
                    sentiment_list.append((first, "Negative"))
                elif v == 0.0:
                    sentiment_list.append((first, "Neutral"))
                else:
                    sentiment_list.append((first, "Positive"))   
    
    return sentiment_list 

In [159]:
sentimentWordsFunct(first_five)

[('некоторые', 'Neutral'),
 ('список любовь', 'Neutral'),
 ('герой р', 'Neutral'),
 ('растрелять правда', 'Neutral'),
 ('правда', 'Neutral'),
 ('воевать недавно', 'Neutral'),
 ('недавно', 'Neutral'),
 ('неплохой репортаж', 'Neutral'),
 ('наградной лист', 'Neutral'),
 ('лист бой', 'Neutral')]